In [1]:
import cv2
import numpy as np
import glob
from PIL import Image
from skimage import morphology
import tifffile
from tqdm import tqdm
#import napari
import pandas
import ntpath
import math
import os
import sys
import matplotlib.pyplot as plt
from scipy.spatial import KDTree, cKDTree
# from google.colab import drive
# drive.mount('/content/gdrive')

In [2]:
chromosome=[]
membrane=[]
nucleous=[]
pattern=[]

In [3]:
# for file in glob.glob("/data/FIBSEM/vrinda/ROI_1/chro_nucl_00_000_650/*.jpg"):
#   chromosome.append(file)

# for file in glob.glob("/data/FIBSEM/vrinda/ROI_1/mem_nucl_00_000_650/*.jpg"):
#   membrane.append(file)

# for file in glob.glob("/data/FIBSEM/vrinda/ROI_1/centroid/*.jpg"):
#   nucleous.append(file)

# for file in glob.glob("/data/FIBSEM/vrinda/ROI_1/patt/*.jpg"):
#   pattern.append(file)


In [4]:
#read segmented chromosome images
for file in glob.glob("/data/FIBSEM/nuclie_align_ROI3_1700/nuclie30/chromosome/*.jpg"):
  chromosome.append(file)

#read segmented membrane images
for file in glob.glob("/data/FIBSEM/nuclie_align_ROI3_1700/nuclie30/chromosome/*.jpg"):
  membrane.append(file)

#read original nuclei images
for file in glob.glob("/data/FIBSEM/nuclie_align_ROI3_1700/nuclie30/*.jpg"):
  nucleous.append(file)

#read segmented SC images
for file in glob.glob("/data/FIBSEM/nuclie_align_ROI3_1700/nuclie30/pattern/*.jpg"):
  pattern.append(file)


In [6]:
#define output folder path
output_path = "/data/FIBSEM/vrinda/ROI_3/nuc_30/test_code/res/s23/dec15_st_t0_b0_mid_580/"
#path to txt file with all debug prints
file_path = output_path + "dec15_roi3_nuc30_st_t0b0mid_580.txt"
if not os.path.exists(output_path):
        from pathlib import Path
        Path(output_path).mkdir(parents=True, exist_ok=True)
        
#set the number of slices to skip from begining in run1(starting from top) and run2(start from bottom)
skip_threshold_top = 0
skip_threshold_bottom = 0


In [7]:
#sort list in ascending order of file names
def sort_list(lst):
  lst = sorted(lst, key = lambda fl : int(ntpath.basename(fl).split('.')[0]))
  return lst

In [8]:
#when you read from the folder, files are not loaded into the list in a sorted manner
chromosome = sort_list(chromosome)
membrane = sort_list(membrane)
nucleous = sort_list(nucleous)
pattern = sort_list(pattern)

# print(len(chromosome))
# print(len(membrane))
# print(len(nucleous))
# print(len(pattern))
# print(chromosome)
# print(membrane)
# print(nucleous)
# print(pattern)


893
893
893
893
['/data/FIBSEM/nuclie_align_ROI3_1700/nuclie30/chromosome/000.jpg', '/data/FIBSEM/nuclie_align_ROI3_1700/nuclie30/chromosome/001.jpg', '/data/FIBSEM/nuclie_align_ROI3_1700/nuclie30/chromosome/002.jpg', '/data/FIBSEM/nuclie_align_ROI3_1700/nuclie30/chromosome/003.jpg', '/data/FIBSEM/nuclie_align_ROI3_1700/nuclie30/chromosome/004.jpg', '/data/FIBSEM/nuclie_align_ROI3_1700/nuclie30/chromosome/005.jpg', '/data/FIBSEM/nuclie_align_ROI3_1700/nuclie30/chromosome/006.jpg', '/data/FIBSEM/nuclie_align_ROI3_1700/nuclie30/chromosome/007.jpg', '/data/FIBSEM/nuclie_align_ROI3_1700/nuclie30/chromosome/008.jpg', '/data/FIBSEM/nuclie_align_ROI3_1700/nuclie30/chromosome/009.jpg', '/data/FIBSEM/nuclie_align_ROI3_1700/nuclie30/chromosome/010.jpg', '/data/FIBSEM/nuclie_align_ROI3_1700/nuclie30/chromosome/011.jpg', '/data/FIBSEM/nuclie_align_ROI3_1700/nuclie30/chromosome/012.jpg', '/data/FIBSEM/nuclie_align_ROI3_1700/nuclie30/chromosome/013.jpg', '/data/FIBSEM/nuclie_align_ROI3_1700/nuclie30

In [11]:
#Some slices might not have any information (blank slice, false positives), this could affect the algorithm
#Set slices that needs to be avoided

# avoid = [43, 50] #roi1 nuc00 n=2
# avoid = [512, 515, 517, 519, 521, 523, 524, 525, 526, 527, 528, 530] #roi3 nuc00 n=12
avoid = []
n = int(input("Enter the total number of slices to avoid : "))
print("Enter the slice numbers to be avoided: ")
for m in range(0, n):
    val = int(input())
    avoid.append(val)
print("Slices to be avoided are", avoid)

Enter the total number of slices to avoid : 0
Enter the slice numbers to be avoided: 
Slices to be avoided are []


In [12]:
score_vol={}
score_vol_by_len={}

run_track_chro_track_global = {}
run_track_pattern_track_global = {}
chro_map_global = {}
combined_chro_map_global = {}
combined_chro_track_global = {}
label_list={}
chro_area_global={}
pattern_map_global={}

threshold = 150  #x-y threshold (distance)
z_threshold = 23 #x threshold (number of slices)
# blob_threshold = 250

global mid # the middle slice, either calculated or edited manually
mid_flag = False

final_track={}  # modify later for global tracking or call fn
chro_centroid_global = {} # modify later for global tracking or call fn
nucl_centroid_global = {} # modify later for global tracking or call fn


Label_Img={}
chro_track_global={}
pattern_track_global={}
pattern_img = {}
pattern_avoid=set()
max_id_c_per_slice=[]

global max_id_c #maximum chromosome count identified
max_id_c=-1

global limit
limit = 6

In [13]:
#morphologically remove small objects lesser than ob_size to clean false positives from the image
def clean_small_obj(im, ob_size):

  cleaned_im = np.copy(im)
  cleaned_index = (cleaned_im!=0)

  img = cleaned_index
  img = morphology.remove_small_objects(img, ob_size, connectivity=1)
  target_img = cleaned_im[:,:]

  cleaned_im[:,:] = np.multiply(target_img, img)

  return cleaned_im

In [14]:
# pass points as tuples
def cal(p1,p2):
  return np.linalg.norm(np.array(p1) - np.array(p2)) # use same metric or try diff?

In [15]:
#cluster chromosomes in the first slice processed and assign labels based on distance threshold
#id_c: chromosome_id, id_b:blob_id, id_s:slice_id
#chro_map - dict, {blob_centroid (k) : [id_c,id_b,id_s] (v)}
def assign_chro_id_first_slice(x, y, id_c, id_b, id_s, cnt, chro_map):
  curr_tag, closest_dist = -1,float('inf')
  if len(chro_map) ==0:
    curr_tag = [id_c,id_b,id_s]
    id_c+=1
    chro_map[(x,y)] = curr_tag
  else:
    closest_pt = None
    for pt,tag in chro_map.items():
      dist = cal(pt,(x,y))
      print((x,y), pt, dist)

      if closest_dist > dist:
        closest_dist,closest_pt = dist,pt

    if closest_dist > threshold: 
      curr_tag= [id_c,id_b,id_s]
      print("new - greater threshold", x,y, id_c)
      id_c+=1
      chro_map[(x,y)] = curr_tag
    else:
      chro_map[(x,y)] = [chro_map[closest_pt][0], id_b, id_s]
      print("less than threshold (chro_map[closest_pt][0])", x,y, closest_pt, chro_map[closest_pt][0], closest_dist)
      # curr_tag = chro_map[closest_pt]
      # x,y=closest_pt
    # id_b+=1
    # print(" cnt : {}, chromosome id : {}, blob id : {}, centroid : {}, length : {}".format(cnt, chro_map[(x,y)][0], chro_map[(x,y)][1], (x,y), sel_cntr_len[i]))
    # print(" closest pt : {}, closest dist: {}".format(closest_pt, closest_dist))
    # print(" ")
  print(chro_map)
  id_b+=1
  return id_c, id_b, id_s, chro_map

In [16]:
def find_closest_centroid(a, b, previous_id_s):
  rank_prev = {}
  prev_chro_centroid = chro_centroid_global[previous_id_s]

  curr_tag, closest_dist = -1,float('inf')
  closest_pt = None

  for id_c_prev, centroid_prev in prev_chro_centroid.items():
    if centroid_prev == 0:
      continue
    x2,y2 = centroid_prev[0], centroid_prev[1]
    dist = cal((a,b), (x2,y2))
    # print("older_slice ",previous_id_s )
    # print((a,b), (x2,y2), dist)
    rank_prev[dist] = [id_c_prev, (x2,y2), previous_id_s]
    # print("rank_prev", rank_prev)

    # if closest_dist > dist:
    #   closest_dist, closest_pt, closest_id = dist, centroid_prev, id_prev
  if(len(rank_prev)>1):
    closest_dist, closest_pt, closest_id, closest_slice_no = find_closest_blob(rank_prev, a, b)
  else:
    first_element = list(rank_prev.keys())[0]
    closest_dist, closest_pt, closest_id, closest_slice_no = first_element, rank_prev[first_element][1], rank_prev[first_element][0], rank_prev[first_element][2]

  return closest_dist, closest_pt, closest_id, closest_slice_no



In [17]:
def find_closest_blob(rank, m, n):

  closest_chromosomes = []

  R=rank
  rank = {k : R[k] for k in sorted(R)}
  print("rank sorted", rank)

  keys = list(rank.keys())

  # for key in rank.keys():
  #   if key < blob_threshold:
  #     closest_chromosomes.append(rank[key][0])
  #   slice_no = rank[0][2]
  closest_chromosomes.append(rank[keys[0]][0])
  closest_chromosomes.append(rank[keys[1]][0])
  if(len(keys)>2):
    closest_chromosomes.append(rank[keys[2]][0])
  slice_no = rank[keys[0]][2]

  curr_tag, closest_blob_dist = -1,float('inf')
  closest_blob, closest_chromosome_id = None, None

  for candidate in closest_chromosomes:
    blob_list = chro_track_global[slice_no][candidate]
    print("chromosome", candidate, blob_list)
    blob_rank={}
    for blob in blob_list:
      blob_dist = cal((m,n), blob)
      blob_rank[blob] = blob_dist
      if closest_blob_dist > blob_dist:
        closest_blob_dist, closest_blob, closest_chromosome_id, closest_slice_no = blob_dist, blob, candidate, slice_no
    print("blob_rank", blob_rank)
  return closest_blob_dist, closest_blob, closest_chromosome_id, closest_slice_no


In [18]:
#cluster chromosomes in the current slice and assign label of the closest blob in the previous slice
def assign_chro_id_rest_slice(x, y, id_b, id_s, cnt, chro_map, direction_flag, n_list, run, start):
  #a dict that store the distance as key and chromosome info as value, for the previous slice
  rank_immediate={}
  # thresh_local = threshold
  thresh_local = threshold - (0.12 * threshold) #reduce threshold to narrow search area as we go backwards

  if direction_flag == False:
    tmp = id_s+1
    while tmp in avoid:
      tmp = tmp+1
  else:
    tmp = id_s-1
    while tmp in avoid:
      tmp = tmp-1

  prev_id_s = tmp
  prev_chro_centroid = chro_centroid_global[prev_id_s]
  flag = False

  curr_tag, closest_dist = -1,float('inf')
  closest_pt = None

  for id_c_prev, centroid_prev in prev_chro_centroid.items():
    if centroid_prev == 0:
      continue
    x2,y2 = centroid_prev[0], centroid_prev[1]
    dist = cal((x,y), (x2,y2))
    # print((x,y), (x2,y2), dist)
    rank_immediate[dist]=[id_c_prev, (x2,y2), prev_id_s]
    # if closest_dist > dist:
    #   closest_dist, closest_pt, closest_id = dist, centroid_prev, id_prev

  print("id_b", id_b, (x,y))
  print("rank_immediate", rank_immediate)

  if(len(rank_immediate)>1):
    closest_dist, closest_pt, closest_id, closest_slice_no = find_closest_blob(rank_immediate, x, y)
  else:
    first_element = list(rank_immediate.keys())[0]
    closest_dist, closest_pt, closest_id, closest_slice_no = first_element, rank_immediate[first_element][1], rank_immediate[first_element][0], rank_immediate[first_element][2]

  closest_dist_immediate_prev_slice, closest_pt_immediate_prev_slice, closest_id_immediate_prev_slice = closest_dist, closest_pt, closest_id

  if closest_dist <= threshold: 
    chro_map[(x,y)] = [closest_id, id_b, id_s]
    print("***********")
    print("Hit in the immediate prev slice (less than threshold)", (x,y), closest_pt, closest_id, closest_dist, closest_slice_no)
    print("***********")
    print(" ")
  else:
    #n_list is a list that stores the extra chromosomes that we found apart from the ones found in the first slice, as we continue the run through slices
    if len(n_list) != 0:
      for id_n_list, centroid_n_list in n_list.items():
        x3,y3 = centroid_n_list[0], centroid_n_list[1]
        dist = cal((x,y), (x3,y3))
        # print((x,y), (x3,y3), dist)

        if closest_dist > dist:
          closest_dist, closest_pt_n, id_n = dist, centroid_n_list, id_n_list
          flag = True
    if(flag == True):
      if closest_dist<= threshold: #210
        chro_map[(x,y)] = [id_n, id_b, id_s]
        print("matched from n_list", x,y, closest_pt_n, id_n, closest_dist)
        return id_b, id_s, chro_map, n_list
    closest = closest_dist

    if(run == 0):
      stop = mid
    else:
      stop = start

    print("stop - ", stop)

    if direction_flag == False:
        end_at = stop+1
    else:
        end_at = stop-1

    count = 1
    start_count = 1
    while(closest > thresh_local): 
      print("thresh_local = {} at  count = {} and prev_id_s = {}".format(thresh_local, count, prev_id_s ))
      # thresh_local = thresh_local - (0.05 * thresh_local)

      # if count == 1:
      #   thresh_local = thresh_local - (0.12 * thresh_local)
      # if (count - start_count) == 10:
      #   thresh_local = thresh_local - (0.12 * thresh_local)
      #   start_count = count



      if direction_flag == False:
        tmpp = prev_id_s+1
        while tmpp in avoid:
          count+=1
          tmpp = tmpp+1

      else:
        tmpp=prev_id_s-1
        while tmpp in avoid:
          tmpp = tmpp-1
          count+=1
      prev_id_s = tmpp
      count+=1

      if (count - start_count) >= 10:
        thresh_local = thresh_local - (0.12 * thresh_local)
        start_count = start_count+10


      # if(prev_id_s == end_at): #############################
      if(count > z_threshold):
        print("Cant find a closest chromosome, mapping to new chromosome")
        global max_id_c
        id = max_id_c + 1
        max_id_c+= 1
        print("new -1", x,y, id)
        print("latest", closest)
        n_list[id]=[x,y]
        print(n_list)
        break
      else:
        if (prev_id_s == end_at):
          print("reached mid before count, mapping to nearest in immediate prev slice")
          print(closest_dist_immediate_prev_slice, closest_pt_immediate_prev_slice, closest_id_immediate_prev_slice)
          id = closest_id_immediate_prev_slice
          break
        else:
          closest, pt, id, slice_no = find_closest_centroid(x, y, prev_id_s)
          # count+=1
          print(count)
          print("point: {} prev_id_s : {} current_id_s: {} closest_id: {} closest point : {} closest_dist: {}".format((x,y),prev_id_s,id_s,id,pt,closest))
    # print("Found closest in a much prev slice")
    # print("point: {} prev_id_s : {} current_id_s: {} closest_id: {} closest point : {} closest_dist: {} slice no of closest : {}".format((x,y),prev_id_s,id_s,id,pt,closest,slice_no))
    chro_map[(x,y)] = [id, id_b, id_s]


  id_b+=1

  return id_b, id_s, chro_map, n_list


In [19]:
#chro_map[(x,y)] = [id_c,id_b,id_s]
#chro_track[id_c] = [(x1,y1),(x2,y2),...] #List of all chromosome blobs labelled as id_c
def get_chromosome_tracking(chro_map):
    chro_track={}

    for pt, tag in chro_map.items():
        if chro_map[pt][0] not in chro_track.keys():
          chro_track[chro_map[pt][0]] = [pt]
        else:
          chro_track[chro_map[pt][0]].append(pt)
    return chro_track

#finding the chromosome centroid as mean of all blob centroids
def find_chro_centroid(chro_track):
    c_centroid = {}

    for id,pts in chro_track.items():

      new_data = pandas.DataFrame(pts)
      c_centroid[id] = list(map(int,new_data.mean(axis=0)))

    # print("len(c_centroid)")
    # print(len(c_centroid))

    return c_centroid

In [20]:
#find nearest membrance point for a particular chromosome centroid
def find_nearest_ngbr(centroid_c, centroid_m, id_s):
  X=centroid_m
  # print(X)
  Y_ds={}
  Y_inds={}
  ctree = cKDTree(X)

  for k in centroid_c.keys():
    if(centroid_c[k]==0):
      continue
    x= centroid_c[k]
    # print(x)
    num_nbrs = num_nbrs = min(50, len(X)) #num_nbrs =50 #when applying org[org<40] = 0 , num_nbrs = min(50, len(X))
    ds, inds =  ctree.query(x, num_nbrs) # finds 30 nearest neighbors,  the hits are sorted by distance (nearest first).
    Y_ds[k]=ds
    Y_inds[k]=inds
  print("len(Y_inds)")
  print(len(Y_inds))
  # print(Y_ds)
  # print(Y_inds)
  # print(Y_ds[0])
  # print(Y_inds[0])
  # print(Y_ds[0][0])
  # print(Y_inds[0][0])
  print(" xxxxxxxxxxxxxxxxxxxxxxxx")

  #shortest distances in the slice for each chro

  dist={}
  closest_dist={}
  index={}
  closest_index={}

  global limit
  limit = max(limit, max_id_c+1)
  for i in range(limit):
    if i in Y_ds.keys():
        avg_of_50_nbrs = np.mean(np.array(Y_ds[i][0:50]))
        dist[i] = avg_of_50_nbrs
        index[i]=[X[Y_inds[i][0]]]
        for k in range(1,num_nbrs):
          index[i].append(X[Y_inds[i][k]])
          closest_dist[i]=Y_ds[i][0]
          closest_index[i]=X[Y_inds[i][0]]
    else:
      dist[i]=0
      index[i]=[0]
  # print("dist", dist)
  # print("index", index)
  print("closest_dist", closest_dist)
  print("closest_pt", closest_index)
  final_track[id_s] = dist
  # print("final_track", final_track)
  return dist, index, closest_dist, closest_index


In [21]:
#for patterns / SC
def join_neighbors(img, itr):

    kernelSize = (3,3)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, kernelSize)
    dilated_img = cv2.dilate(img,kernel,iterations = itr)

    eroded_img = cv2.erode(dilated_img,kernel,iterations = itr)

    return eroded_img

In [22]:
def find_contours(im):

  contours, hierarchy = cv2.findContours(image=im, mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_NONE)
  mask = np.zeros(im.shape, np.uint8)
  # contours =list(contours)
  # cv2.drawContours(mask,contours,-1,(255,255,255))
  return contours, hierarchy, mask

In [23]:
def load_image_and_find_contour(image_id, image_list, clean_flag, obj_size):
  org = Image.open(image_list[image_id])
  org.load()
  org = np.array(org)

  if(clean_flag == True):
    org = clean_small_obj(org,obj_size)

  org[org<40]=0 #images are jpeg, avoiding the slightly gradient pixels outside the main boundary of the blobs

  contours_org, hierarchy_org, mask_org = find_contours(org)

  # mask_c3 = np.stack((mask_c,mask_c,mask_c), axis=2)
  # mask_m3 = np.stack((mask_m,mask_m,mask_m), axis=2)
  # mask_added = cv2.addWeighted(mask_c3, 1, mask_m3, 0.7, 0)
  # mask_added = cv2.addWeighted(mask_c, 1, mask_m, 0.7, 0)

  return contours_org, hierarchy_org, mask_org

In [24]:
def cluster_chromosome(contours_c, hierarchy_c, mask_both, mid_flag, id_c, id_b, id_s, chro_map_local, direction_flag, run, start):
  new_list={}
  mask_label= np.zeros(mask_both.shape, np.uint8)
  for i in range(len(contours_c)):
    if cv2.arcLength(contours_c[i],True) > 25 and hierarchy_c[0][i][3]==-1:
      # ccontr.append(contours_c[i])
      # sel_cntr_len[i]=cv2.arcLength(contours_c[i],True)
      M = cv2.moments(contours_c[i])
      if(M["m00"])==0:  #avoid division by zero
        continue
      cX,cY = int(M["m10"] / M["m00"]),int(M["m01"] / M["m00"])
      # centroids_chro_blobs.append([id_b,cX,cY])
      if(mid_flag==True):  #if mid_flag==True first slice is not processed yet
        id_c, id_b, id_s, chro_map_local = assign_chro_id_first_slice(cX, cY, id_c, id_b, id_s, i, chro_map_local)
      else:
        id_b, id_s, chro_map_local, new_list= assign_chro_id_rest_slice(cX, cY, id_b, id_s, i, chro_map_local, direction_flag, new_list, run, start)

      points=[]
      for l in range(len(contours_c[i])):
        points.append(contours_c[i][l][0])
      points = np.array(points)
      # cv2.drawContours(mask_label,[points],-1,label[chro_map_local[(cX,cY)][0]], thickness = cv2.FILLED)
      #draw filled contour with label assigned to the blob on a mask for the slice
      cv2.drawContours(mask_label,[points],-1,((chro_map_local[(cX,cY)][0])+1), thickness = cv2.FILLED)

      #to calculate total area of chromosomes
      area = cv2.contourArea(contours_c[i])
      chro_map_local[(cX,cY)].append(area)

  Label_Img[id_s]=mask_label #stores the labelled (filled contours with chromosome label) masks for each slice
  # Chro_Im_track.append([mask_both, id_s])

  if(mid_flag==True):
       return id_c, id_b, id_s, chro_map_local
  else:
      return id_b, id_s, chro_map_local

In [25]:
#find center of nucleous in each slice from original image
def get_nucl_centroids(contours_n):
  for i in range(len(contours_n)):
    if cv2.arcLength(contours_n[i],True) > 4000 :
      M = cv2.moments(contours_n[i])
      if M['m00']!=0.0:
        cX,cY = int(M["m10"] / M["m00"]),int(M["m01"] / M["m00"])

  return (cX,cY)


In [26]:
def find_chromosome(start, end, mid_flag, direction_flag, run): #direction_flag = True --> fwd (mid-end), = False -->bckwd (mid-beg)
    # if (mid_flag != True):
    #   if(direction_flag == False):
    #     start = start-1
    j = start
    id_s = start
    while(j != end):

      print(j)
      print(chromosome[j])
      print(membrane[j])

      contours_c, hierarchy_c, mask_c = load_image_and_find_contour(j, chromosome, True, 800)
      contours_m, hierarchy_m, mask_m = load_image_and_find_contour(j, membrane, False, 0)
      contours_n, hierarchy_n, mask_n = load_image_and_find_contour(j, nucleous, False, 0)
      mask_both = cv2.addWeighted(mask_c, 1, mask_m, 0.7, 0)

      n = ntpath.basename(chromosome[j]).split('.')[0]
      n=int(n)
      if n in avoid:
        print(n)
        final_track[j]='inf'
        # Img_col_centroids[id_s]=mask_both
        # Img_col[id_s]=mask_both
        # Label_Img_centroid[id_s]=mask_both
        Label_Img[id_s]=mask_both
        # Chro_Im_track.append([mask_both, id_s])
        chro_centroid_global[id_s] = 'inf'
        nucl_centroid_global[id_s] = 'inf'
        chro_track_global[id_s] = 'inf'
        chro_map_global[id_s] ='inf'
        chro_area_global[id_s] ='inf'
        max_id_c_per_slice.append([id_s, 'inf'])
        if(direction_flag == True):
          id_s+=1
          j+=1
        else:
          id_s-=1
          j-=1
        continue

      id_c=0
      id_b=0

      # nucl_centroid_global[id_s] = get_nucl_centroids(contours_n)
      # print("nucl_centroid_global :", nucl_centroid_global)

      chro_map_local={}
      chro_track_local={}
      chro_centroid_local={}

      # centroids_chro_blobs=[] #add slice wise as well later
      # ccontr=[]
      # sel_cntr_len={}

      if(mid_flag==True):
        id_c, id_b, id_s, chro_map_local = cluster_chromosome(contours_c, hierarchy_c, mask_both, mid_flag, id_c, id_b, id_s, chro_map_local, direction_flag, run, start)
        mid_flag = False
      else:
        id_b, id_s, chro_map_local = cluster_chromosome(contours_c, hierarchy_c, mask_both, mid_flag, id_c, id_b, id_s, chro_map_local, direction_flag, run, start)

      print("heyyyyyyyyyyy")
      print("chro_map_local : ",chro_map_local)

      #for global tracking of chromosomes over entire stack
      for point,tag in chro_map_local.items():
        temp = {point: tag[0]}
        temp_area = {point: tag[3]}
        if j in chro_map_global:
          chro_map_global[j][point]=tag[0]
        else:
          chro_map_global[j] = temp

        if tag[0] not in label_list:
          label_list[tag[0]] = None

        if j in chro_area_global:
          chro_area_global[j][point]=tag[3]
        else:
          chro_area_global[j] = temp_area

      chro_track_local = get_chromosome_tracking(chro_map_local)
      print("chro_track_local :", chro_track_local)

      chro_track_global[id_s] = chro_track_local
      # print("chro_track_global: ", chro_track_global)

      global max_id_c
      max_id_c = max(max_id_c, id_c-1)
      print("max_id_c_till_now: ", max_id_c)

      max_id_c_per_slice.append([id_s, len(chro_track_local)])
      print("max_chrmosome_per_slice: ", max_id_c_per_slice)

      if len(chro_track_local)==0:
        avoid.append(id_s)
        print(avoid)

      chro_centroid_local = find_chro_centroid(chro_track_local)
      print("chro_centroid_local_before: ",chro_centroid_local)

      chro_centroid_global[id_s] = chro_centroid_local
      # print("chro_centroid_global: ", chro_centroid_global)

      # Img_col[id_s] = mask_both

      centroids_mem=[]
      mcontr=[]
      sel_mcntr_len={}
      for i in range(len(contours_m)):
        if cv2.arcLength(contours_m[i],True) > 0 :
          M = cv2.moments(contours_m[i])
          if M['m00']!=0.0:
            mcontr.append(contours_m[i])
            sel_mcntr_len[i]=cv2.arcLength(contours_m[i],True)
            cX,cY = int(M["m10"] / M["m00"]),int(M["m01"] / M["m00"])
            centroids_mem.append([cX,cY])

      list_dis, list_index, nearest_dist, nearest_pt = find_nearest_ngbr(chro_centroid_global[id_s], centroids_mem, id_s)

      mask_both_temp = np.copy(mask_both)
      # generate_mapped_images(chro_centroid_global[id_s], list_index, mask_both_temp, id_s)

      order_id.append(id_s)

      # if(id_s==100):
      #   break
      if(direction_flag == True):
        id_s+=1
        j+=1
      else:
        id_s-=1
        j-=1

      print("___________________________________________________________")

In [27]:
def sort_dictionary(unsorted_dictionary):
  Y=unsorted_dictionary
  sorted_dictionary = {k : Y[k] for k in sorted(Y)}
  print(sorted_dictionary.keys())
  return sorted_dictionary

In [28]:
# def save_label_stack(Label_Img, run, save_patterns_flag):
#   # path = "/content/gdrive/MyDrive/roi_1_nucl_00/test_code/res/15_dec/full"
#   # path = "/content/gdrive/MyDrive/roi3_nuc00/test/res/23_jan/full"
#   # path = "/content/gdrive/MyDrive/roi_1_nucl_00/test_code/res/18_feb_th_150_z_23_trial_top_bott/full_"
# #   path = "/data/FIBSEM/vrinda/ROI_1/test_code/res/Apr_6/test/"
#   path = "/data/FIBSEM/vrinda/ROI_2/test_code/res/nuc00/Apr_6/test/"
#   # for i in Img_col_centroids.keys():
#   if save_patterns_flag == True:
#     folder_name = "patterns"
#   else:
#     folder_name = "chromosome"
#   run_tag = "run_"+str(run)+"/"+folder_name+"_all_labels_"+"run_"+str(run)
#   path = path + run_tag
#   for i in Label_Img.keys():
#     tag=i
#     if not os.path.exists(path):
#       from pathlib import Path
#       Path(path).mkdir(parents=True, exist_ok=True)
#     # cv2.imwrite(path+"/"+ f"{tag:03d}"+".jpg", (Img_col_centroids[i]))
#     cv2.imwrite(path+"/"+ f"{tag:03d}"+".jpg", (Label_Img[i]))

#      mask_old_path = "/data/FIBSEM/vrinda/ROI_1/test_code/hand_segment/output/nuc_00/try2/19/"+'mask_less_than_40.tiff'

In [29]:
#save masks with all labels to the system for each slice for entire stack
def save_label_stack(Label_Img, run, save_patterns_flag):
    # path = "/content/gdrive/MyDrive/roi_1_nucl_00/test_code/res/15_dec/full"
    # path = "/content/gdrive/MyDrive/roi3_nuc00/test/res/23_jan/full"
    # path = "/content/gdrive/MyDrive/roi_1_nucl_00/test_code/res/18_feb_th_150_z_23_trial_top_bott/full_"
#     path = "/data/FIBSEM/vrinda/ROI_1/test_code/res/Apr_22/test/"
#     path = "/data/FIBSEM/vrinda/ROI_2/test_code/res/nuc00/Apr_6/test/"
    # for i in Img_col_centroids.keys():
    if save_patterns_flag == True:
        folder_name = "patterns"
    else:
        folder_name = "chromosome"
    run_tag = "run_"+str(run)
    path = output_path + run_tag
    if not os.path.exists(path):
      from pathlib import Path
      Path(path).mkdir(parents=True, exist_ok=True)
    img_list=[]
    for i in Label_Img.keys():
        img_list.append(Label_Img[i])
    new_path = path+"/"+folder_name+"_all_labels_"+"run_"+str(run)+".tiff"
    # cv2.imwrite(path+"/"+ f"{tag:03d}"+".jpg", (Img_col_centroids[i]))
#     cv2.imwrite(path+"/"+ f"{tag:03d}"+".jpg", (Label_Img[i]))
    img_list = np.array(img_list)
    tifffile.imwrite(new_path, img_list)




In [30]:
#saves individual chromosomes or SC's into individual folders
def save_individual_chromosome(Label_Img, chromosome_count, chro_to_avoid, run, save_patterns_flag):
  # path2="/content/gdrive/MyDrive/roi_1_nucl_00/test_code/res/15_dec/org/"
  # path2="//content/gdrive/MyDrive/roi3_nuc00/test/res/23_dec_2/org/"
  # path2 = "/content/gdrive/MyDrive/roi_1_nucl_00/test_code/res/18_feb_th_150_z_23_trial_top_bott/org/"
#   path2 = "/data/FIBSEM/vrinda/ROI_1/test_code/res/Apr_22/test/"
#   path2 = "/data/FIBSEM/vrinda/ROI_2/test_code/res/nuc00/Apr_6/test/"
  if save_patterns_flag == True:
    folder_name = "patterns"
  else:
    folder_name = "chromosome"
  run_tag = "run_"+str(run)+"/individual/"+folder_name
  path2 = output_path + run_tag +"/"
  for h in chromosome_count:
    if h in chro_to_avoid:
      continue
    label_chro = h+1
    for k in range(len(chromosome)):
      temp = (Label_Img[k]==label_chro)
      temp = temp.astype(np.uint8)
      if save_patterns_flag == True:
          chro_temp = Image.open(patterns[k])
      else:
          chro_temp = Image.open(chromosome[k])
      chro_temp.load()
      chro_temp = np.array(chro_temp)
      chro_temp[chro_temp<40]=0
      temp3= cv2.bitwise_and(chro_temp,chro_temp, mask = temp)
      temp3[temp3>0] = 1  
      nuc_temp = Image.open(nucleous[k])
      nuc_temp.load()
      nuc_temp = np.array(nuc_temp)
      temp2= cv2.bitwise_and(nuc_temp,nuc_temp, mask = temp3)
      tag=k
      if save_patterns_flag == True:
        path_tag = "patt_"+str(label_chro)
      else:
        path_tag = "ch_"+str(label_chro)
      complete_path = path2+path_tag
      if not os.path.exists(complete_path):
        from pathlib import Path
        Path(complete_path).mkdir(parents=True, exist_ok=True)
      # cv2.imwrite(complete_path+"/"+ f"{tag:03d}"+".jpg", (temp2* 255).astype(np.uint8)) #####
      cv2.imwrite(complete_path+"/"+ f"{tag:03d}"+".jpg", (temp2))
    print(label_chro)


In [31]:
#find centroid of patterns/SC
def find_pattern_centroids(patt_img, chromosome_count, chro_to_avoid):
  for h in range(len(patt_img)):
    if h in avoid:
        continue
    if h in pattern_avoid:
        continue
    pattern_map_local={}
    for pq in chromosome_count:
        if pq in chro_to_avoid:
          continue
        label_patt = pq+1
        img_temp4 = np.copy(patt_img[h]==label_patt)
        img_temp4 = img_temp4.astype(np.uint8)
        img_temp4 = join_neighbors(img_temp4, 50)
        cont, hier, mas = find_contours(img_temp4)
        for i in range(len(cont)):
            if cv2.arcLength(cont[i],True) > 25 :
              M = cv2.moments(cont[i])
              if M["m00"] == 0:
                continue
              cX,cY = int(M["m10"] / M["m00"]),int(M["m01"] / M["m00"])
              pattern_map_local[(cX,cY)]= [pq]
              if img_temp4[cY,cX] == 0:
                  print(img_temp4[cY,cX], (cY,cX), h, pq)
#     print("pattern_map_local :", map_local)
    pattern_track_local = get_chromosome_tracking(pattern_map_local)
#     print("pattern_track_local :", track_local)

    pattern_map_global[h]=pattern_map_local
    pattern_track_global[h] = pattern_track_local
  # print("pattern_track_global :", pattern_track_global)

In [32]:
def find_pattern_label(X,Y,chro_blobs_in_the_slice):
  closest_id, closest_dist = -1,float('inf')
  for pt,id in chro_blobs_in_the_slice.items():
    # if centroid_prev == 0:
    #   continue
    dist = cal((X,Y), pt)
    print((X,Y), pt, dist)
    if closest_dist > dist:
        closest_dist, closest_pt, closest_id = dist,pt,id
  # if closest_dist > threshold_patt:
  #   return -1
  # else:
  #   return closest_id
  return closest_id

In [33]:
#identify SC blobs near to each chromosome and assign same label as chromosome based on distance
def find_patterns(run, chro_map, chromosome_count, chro_to_avoid):
  print("pattern_track_global :", pattern_track_global)
  zero_moment=[]
  for k in range(len(pattern)):
    print(k)
    contours_p, mask_p = load_image_and_find_contour(k, pattern, True, 600)
    n = ntpath.basename(pattern[k]).split('.')[0]
    n=int(n)
    if n in avoid:
      print(n)
      pattern_img[k] = mask_p
      # pattern_avoid.append(k)
      pattern_avoid.add(k)
      continue
    if len(contours_p)==0:
      # pattern_avoid.append(k)
      pattern_avoid.add(k)
      pattern_img[k] = mask_p
      continue
    pattern_map_inside_find_patterns = {}
    blobs_in_slice = chro_map[k]
    print("blobs_in_slice", blobs_in_slice)
    for i in range(len(contours_p)):
      if cv2.arcLength(contours_p[i],True) > 25 :
        M = cv2.moments(contours_p[i])
        if M["m00"] == 0:
          zero_moment.append(k)
          continue
        cX,cY = int(M["m10"] / M["m00"]),int(M["m01"] / M["m00"])
        patt_label = find_pattern_label(cX,cY, blobs_in_slice)
        print("patt_label", (cX,cY), patt_label)
        pattern_map_inside_find_patterns[(cX,cY)] = [patt_label]
        points_new=[]
        for l in range(len(contours_p[i])):
          points_new.append(contours_p[i][l][0])
        points_new = np.array(points_new)
        cv2.drawContours(mask_p,[points_new],-1,patt_label+1, thickness = cv2.FILLED)
    pattern_img[k]=mask_p

    print("pattern_map_inside_find_patterns :", pattern_map_inside_find_patterns)
    # pattern_track_local = get_chromosome_tracking(pattern_map_local)
    # print("pattern_track_local :", pattern_track_local)

    # pattern_track_global[k] = pattern_track_local ######change names
    # pattern_map_global[k]=pattern_map_inside_find_patterns
  print("zero_moment", zero_moment)
  # sort_dictionary(Final_Image_Stack)
  # print(Final_Image_Stack.keys())
#   save_label_stack(pattern_img, run, True)
#   save_individual_chromosome(pattern_img, chromosome_count, chro_to_avoid, run, True)
#############################################################################################################
  find_pattern_centroids(pattern_img, chromosome_count, chro_to_avoid)
  print("pattern_track_global :", pattern_track_global)

In [34]:
#used in length calculation of SC
#to find next closest point in that particular leg of SC
#search in a rectangle around the point in coming slices to avoid finding a point from differnt leg
def find_closest_leg_point(x,y,z, pattern_in_current_slice_final, processed_in_current_slice):
  if False not in processed_in_current_slice.values():
    return None, None, None, processed_in_current_slice
  closest_x_temp, closest_x_diff, closest_point_temp = None, float('inf'), None
  point_list=[]

  for i in range(len(pattern_in_current_slice_final)):
    current_point = pattern_in_current_slice_final[i]
    if processed_in_current_slice[current_point]==True:
      continue
    (a1,b1,c1) = current_point
    x_diff = abs(x-a1)
    y_diff = abs(y-b1)
    if(x_diff<=25):
      if(y_diff<=25):
        point_list.append([x_diff, y_diff, (a1,b1,c1)])

#   print("point_list b4", point_list)
  point_list = sorted(point_list, key = lambda x : (x[0], x[1]))
  print("point_list", point_list)

  if point_list:
    closest_point_temp = point_list[0][2]
    closest_x_diff = point_list[0][0]
    closest_y_diff = point_list[0][1]
    processed_in_current_slice[closest_point_temp]=True
    print("closest_point", point_list[0])
    return closest_point_temp[0], closest_x_diff, closest_point_temp, processed_in_current_slice

  else:
    return None, None, None, processed_in_current_slice


In [35]:
#calculate length of SC
def find_length(pattern_track, chromosome_count, chro_to_avoid):

  print("chromosome_count, chro_to_avoid", chromosome_count, chro_to_avoid)
#   print("pattern_track", pattern_track)

  length = {}
  missing_patt = {}
  begining_pt_track={}
  ending_pt_track={}
  legs_length_track={}

  for c in chromosome_count:
    print(" ")
    print("c", c)
    # if c==2:
    #   break
    legs_length={}
    begining_pt = {}
    ending_pt={}
    missing_leg={}
    legs_track_for_chromosome = {}
    found_next_for_chromosome = {}
    leg_count=1
    if c in chro_to_avoid:
      continue
    length[c]=0
    missing_patt[c] = []
    first_presence = 0
    found_first_presence = False
    for slice_id in range(len(pattern)):
      print("slice_id", slice_id)
      if slice_id in pattern_avoid:
        print("yes")
        continue
      if c not in pattern_track[slice_id]:
        missing_patt[c].append(slice_id)
        print("yes22")
        continue
      else:
        first_presence = slice_id
        print("yes333")
        found_first_presence = True
        break
    if found_first_presence == False:
        length[c]=0
        continue
    print("first_presence", first_presence)
    print("pattern_track_global[first_presence]", pattern_track[first_presence])
    patterns_in_first_presence = pattern_track[first_presence][c]
#     print("patterns_in_first_presence", patterns_in_first_presence)
    n_legs = len(patterns_in_first_presence)
    print(" n_legs ", n_legs)
    found_next_for_legs={}
    legs_track_for_legs = {}
    for point in patterns_in_first_presence:
      (m,n) = point
      legs_length[leg_count] = 0
      begining_pt[leg_count] = (m,n,slice_id)
      ending_pt[leg_count] = (m,n,slice_id)
      missing_leg[leg_count] = 0
      found_next_for_legs[leg_count] = False
      legs_track_for_legs[leg_count] = (m,n,slice_id)
#       print("legs_length", legs_length)
#       print("begining_pt", begining_pt)
#       print("found_next_for_legs for ch {} is {}".format(c,found_next_for_legs))
#       print("legs_track_for_legs for ch {} is {}".format(c,legs_track_for_legs))
      leg_count+=1

    found_next_for_chromosome[first_presence] = found_next_for_legs
#     print("found_next_for_chromosome for ch {} is {}".format(c,found_next_for_chromosome)) #{slice number:{leg number: T/F}}
    legs_track_for_chromosome[first_presence] = legs_track_for_legs
#     print("legs_track_for_chromosome for ch {} is {}".format(c,legs_track_for_chromosome)) #{slice number:{leg number: POINTS}}
#     print(" ")
    #######################################################
    for slice_id in range(first_presence+1, len(pattern)):
      found_next_for_legs={}
      legs_track_for_legs = {}
      print(" ")
      print("slice_id", slice_id)
      # if slice_id == first_presence+35:
      #   break
      if slice_id in pattern_avoid:
        continue
      if c not in pattern_track[slice_id]:
#         print("missing in ", slice_id)
        missing_patt[c].append(slice_id)
        continue
      pattern_in_current_slice_temp = pattern_track[slice_id][c]
      print("pattern_track_global[slice_id = {}][c = {}] {}".format(slice_id, c, pattern_in_current_slice_temp))
      pattern_in_current_slice_temp = sorted(pattern_in_current_slice_temp, key = lambda x : (x[0], x[1]))
      print("pattern_in_current_slice_temp", pattern_in_current_slice_temp)
      processed_in_current_slice={}
      pattern_in_current_slice_final=[]
      for point in pattern_in_current_slice_temp:
        (m,n) = point
        pattern_in_current_slice_final.append((m,n,slice_id))
        processed_in_current_slice[(m,n,slice_id)]=False
#       print("pattern_in_current_slice_final", pattern_in_current_slice_final)
#       print("processed_in_current_slice", processed_in_current_slice)

      prev_slice = slice_id-1
      while ((prev_slice in pattern_avoid) or (prev_slice in missing_patt[c])):
        prev_slice-=1
        if prev_slice == -1: #comment?
          break
      if prev_slice == -1:
          break
      # print("prev_slice", prev_slice)

      # while prev_slice in missing_patt[c]:
      #   prev_slice-=1
      #   if prev_slice == -1: #comment?
      #     break
      print("prev_slice after looking in pattern_avoid and missing patt", prev_slice)

      legs_in_prev_slice = legs_track_for_chromosome[prev_slice]
      print("legs_in_prev_slice", legs_in_prev_slice)

      for key in found_next_for_chromosome[prev_slice].keys():
        found_next_for_legs[key] = False

      for leg in legs_in_prev_slice:
#         print("leg", leg)
#         print("missing_leg", missing_leg)
        if found_next_for_chromosome[prev_slice][leg] == False and missing_leg[leg]<=25:
          if leg not in legs_in_prev_slice:
            (x1,y1,z1) = ending_pt[leg]
#             print("@@@@@@@@@@@@@@@@@@@@@@leg_not_there")
          else:
            (x1,y1,z1) = legs_in_prev_slice[leg]
          print("(x1,y1,z1)", (x1,y1,z1))
          # closest_x, closest_diff, closest_point, processed_in_current_slice = find_closest_leg_point(876,1070,1, pattern_in_current_slice_final, processed_in_current_slice)
          closest_x, closest_diff, closest_point, processed_in_current_slice = find_closest_leg_point(x1,y1,z1, pattern_in_current_slice_final, processed_in_current_slice)
          if closest_point !=None:
            length_temp = cal(legs_in_prev_slice[leg], closest_point)
            legs_length[leg] += length_temp
            ending_pt[leg] = closest_point
            found_next_for_chromosome[prev_slice][leg] = True####
#             print("######found_next_for_chromosome for ch {} is {}".format(c,found_next_for_chromosome)) #{slice number:{leg number: T/F}}
            found_next_for_legs[leg] = False ####????
            legs_track_for_legs[leg] = closest_point
            missing_leg[leg]=0
          else:
            missing_leg[leg]+=1
            legs_track_for_legs[leg] = ending_pt[leg]
            print("leg", leg)
            print("closest_point_None")
            print("slice_id", slice_id)
            print("prev_slice", prev_slice)
      for pts in processed_in_current_slice.keys():
        if processed_in_current_slice[pts]==False:
          # (m,n,l) = pts
          print(pts)
#           print("adding new leg")
          legs_length[leg_count] = 0
          begining_pt[leg_count] = pts #(m,n,slice_id)
          ending_pt[leg_count] = pts
          missing_leg[leg_count] = 0
          found_next_for_legs[leg_count] = False ####????
          legs_track_for_legs[leg_count] = pts #(m,n,slice_id)
#           print("legs_length", legs_length)
#           print("begining_pt", begining_pt)
#           print("found_next_for_legs for ch {} is {}".format(c,found_next_for_legs))
#           print("legs_track_for_legs for ch {} is {}".format(c,legs_track_for_legs))
          leg_count+=1
#       print("hoyyyy", slice_id)
#       print("found_next_for_chromosome for ch {} is {}".format(c,found_next_for_chromosome)) #{slice number:{leg number: T/F}}
      found_next_for_chromosome[slice_id] = found_next_for_legs
#       print("found_next_for_chromosome for ch {} is {}".format(c,found_next_for_chromosome)) #{slice number:{leg number: T/F}}
      legs_track_for_chromosome[slice_id]=legs_track_for_legs
#       print("legs_track_for_chromosome for ch {} is {}".format(c,legs_track_for_chromosome)) #{slice number:{leg number: POINTS}}
#       print("legs_track_for_chromosome for ch {} for slice {} is {}".format(c,slice_id,legs_track_for_chromosome[slice_id])) #{slice number:{leg number: POINTS}}
#       print("ending_pt", ending_pt)
#       print("missing_leg", missing_leg)
#       print("legs_length", legs_length)
      print(" ")

    total_length_of_patt = 0
    legs_length_track[c]=legs_length

    for l in legs_length:
      total_length_of_patt+=legs_length[l]

    length[c]=total_length_of_patt
    begining_pt_track[c]=begining_pt
    ending_pt_track[c]=ending_pt

  print("missing_patt", missing_patt)
  print("begining_pt_track", begining_pt_track)
  print("ending_pt_track", ending_pt_track)

#   print("length", length)
  length = sort_dictionary(length)
  print("length", length)

  return length, legs_length_track, begining_pt_track, ending_pt_track


In [36]:
#calculate volume of chromosome
def find_volume(chromosome_track, chromosome_area, chromosome_count, chro_to_avoid):
  vol = {}
  for c in chromosome_count:
    if c in chro_to_avoid:
      continue
    vol[c]=0
  for k in range(len(chromosome_track)):
    if k in avoid:
      continue
    temp_track = chromosome_track[k]
    temp_area = chromosome_area[k]
    for chromosome, blob_list in temp_track.items():
      if chromosome in chro_to_avoid:
        continue
      for blob in blob_list:
        vol[chromosome]+= temp_area[blob]

#   print("volume", vol)
  vol = sort_dictionary(vol)
  print("volume", vol)
  return vol

In [37]:
# def find_score(chromosome_track, chromosome_area, chromosome_count, chro_to_avoid):
#   volume = find_volume(chromosome_track, chromosome_area, chromosome_count, chro_to_avoid)
#   vol_list = []
#   for c,v in volume.items():
#     vol_list.append(v)
#   print("vol_list", vol_list)
#   vol_list = np.array(vol_list)
#   print("vol_list", vol_list)
#   score_var = np.var(vol_list)
#   score_std = np.std(vol_list)
#   print("score.var.........", score_var)
#   print("score.std.........", score_std)
#   return score_std

In [38]:
#calculate score of the algorithm
#std.dev(V/l)
def find_score(chromosome_track, chromosome_area, pattern_track, chromosome_count, chro_to_avoid):

  length, legs_length_track, begining_pt_track, ending_pt_track = find_length(pattern_track, chromosome_count, chro_to_avoid)
  volume = find_volume(chromosome_track, chromosome_area, chromosome_count, chro_to_avoid)

  vol_list = []
  len_list = []
  vol_by_len_list = []

  c_len_zero=[]

  for c,v in volume.items():
    vol_c = v
    len_c = length[c]

    if len_c != 0:
        vol_list.append(vol_c)
        len_list.append(len_c)

        v_by_l = vol_c/len_c
        vol_by_len_list.append(v_by_l)
    else:
        c_len_zero.append(c)

  print(" ")
  print("chromosomes avoided due to zero length : ", c_len_zero)

  print(" ")
  vol_list = np.array(vol_list)
  print("vol_list", vol_list)
  len_list = np.array(len_list)
  print("len_list", len_list)
  vol_list = np.array(vol_list)
  print("vol_by_len_list", vol_by_len_list)

  print(" ")
  score_var_vol = np.var(vol_list)
  score_std_vol = np.std(vol_list)
  print("volume.....score.variance.........", score_var_vol)
  print("volume.....score.standard.deviation........", score_std_vol)
  print(" ")

  score_var_v_by_l = np.var(vol_by_len_list)
  score_std_v_by_l  = np.std(vol_by_len_list)
  print("volume_by_length.....score.variance.........", score_var_v_by_l )
  print("volume_by_length.....score.standard.deviation.........", score_std_v_by_l )


  return score_std_vol, score_std_v_by_l, length, volume, legs_length_track, begining_pt_track, ending_pt_track

In [39]:
#count the number of slices the chromosome is present
def count_presence(ch_track):
  slice_count = {}
  for d in ch_track.keys():
    # print("d", d)
    if d in avoid:
      continue
    count_list_temp = ch_track[d]
    # print("count_list_temp", count_list_temp)
    for id in count_list_temp.keys():
      if id in slice_count:
        slice_count[id]+=1
      else:
        slice_count[id]=1
  return slice_count

In [40]:
#find chromosomes which are present in less than 5 slices
def find_short_chromosomes(chro_track_list):
  chro_slice_count = count_presence(chro_track_list)
  print("chro_slice_count", chro_slice_count)
  chromosome_to_be_removed = []
  min_slice_threshold = 20 #5
  for chro_id in chro_slice_count:
    if (chro_slice_count[chro_id] < min_slice_threshold):
      chromosome_to_be_removed.append(chro_id)
  return chromosome_to_be_removed

In [41]:
#run0 mid to bottom and mid to top

global mid
# mid = 357 #roi1 nuc00
# mid = 305 #roi3 nuc00
mid_flag = True
beg = 0
last = len(chromosome)-1

print("beg", beg)
print("last", last)

run = 0 #starting from mid

print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")
print(" RUN - ", run)
print("starting from mid")

mid_temp = beg + ((last-beg)//2)
print("Calculated middle slice is : ", mid_temp)
flag_ip = False
while(flag_ip==False):
  ch = str(input("Proceed with the calculated middle slice (y/n) : "))
  global mid
  if(ch == 'y'):
    mid = mid_temp
    flag_ip = True
  elif(ch == 'n'):
    try:
      slice_no = int(input("Enter the slice number to be considered as reference middle slice : "))
      mid = slice_no
      flag_ip = True
    except:
      print("!!!!!!Please enter a valid slice number!!!!!!")
      continue
  else:
    print("invalid input, please provide a valid input")
print(" ")
print("Starting Slice Number = {}, Ending Slice Number = {}, Middle Reference Slice Number = {}".format(beg, last, mid))

# id_s=mid
id_c=0
id_b=0
chro_map_local={} #??????
print(mid)
print(chromosome[mid])
print(membrane[mid])

order_id = []
# file_path = output_path + "oct_22_roi_2_nuc00.txt"
sys.stdout = open(file_path, "w")

# sys.stdout = open("/content/gdrive/MyDrive/roi_1_nucl_00/test/test2/out.txt", "w")
# sys.stdout = open("/data/FIBSEM/vrinda/ROI_1/test_code/res/Apr_22_test_roi_1_n0.txt", "w")
# sys.stdout = open("/data/FIBSEM/vrinda/ROI_2/test_code/res/nuc00/Apr_20_test_roi_1_n0.txt", "w")



find_chromosome(mid, last+1, mid_flag, True, run)
mid_flag = False
find_chromosome(mid-1, beg-1, mid_flag, False, run)

print("order_id:",order_id)
print(limit)
print(max_id_c)

chro_map_run_0 = chro_map_global
print("*****chro_map_run_0 : ", chro_map_run_0)
chro_area_run_0 = chro_area_global
print("*****chro_area_run_0 : ", chro_area_run_0)
label_list_run0 = label_list
print("*****label_list_run0 : ", label_list_run0)
run_track_chro_track_global[run] = chro_track_global

uniqueValues_0 = set(label_list_run0.keys())
print("uniqueValues",uniqueValues_0)
# new_limit_0 = len(uniqueValues)
# print("new_limit_0", new_limit_0)
chromosomes_to_avoid_run_0 = find_short_chromosomes(run_track_chro_track_global[run])
empty_list = []
print("chromosomes_to_avoid_run_0", chromosomes_to_avoid_run_0)


Label_Img = sort_dictionary(Label_Img)
# save_label_stack(Label_Img, run, False)
save_individual_chromosome(Label_Img, uniqueValues_0, chromosomes_to_avoid_run_0, run, False) #pass empty list intentionally to save all chromosomes

# find_patterns(run, chro_map_run_0, uniqueValues_0, chromosomes_to_avoid_run_0) #finds patterns, saves patterns, finds new pattern centroids
# run_track_pattern_track_global[run] = pattern_track_global
# print("pattern_avoid", pattern_avoid)
# #add score
# # length = find_length(pattern_track_global, new_limit_0, chromosomes_to_avoid_run_0)
# score_v_run0, score_v_by_l_run0, length_run0, volume_run0, legs_length_track_run0, begining_pt_track_run0, ending_pt_track_run0 = find_score(run_track_chro_track_global[run],
#                                                                                                                                              chro_area_run_0, pattern_track_global, uniqueValues_0,
#                                                                                                                                              chromosomes_to_avoid_run_0)
# # score_run0 = find_score(run_track_chro_track_global[run], chro_area_run_0, new_limit_0, chromosomes_to_avoid_run_0)
# score_vol[run] = score_v_run0
# score_vol_by_len[run] = score_v_by_l_run0

print("DONEEEEEE RUN - ", run)

beg 0
last 892
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
 RUN -  0
starting from mid
Calculated middle slice is :  446
Proceed with the calculated middle slice (y/n) : 
invalid input, please provide a valid input
Proceed with the calculated middle slice (y/n) : 
invalid input, please provide a valid input
Proceed with the calculated middle slice (y/n) : n
Enter the slice number to be considered as reference middle slice : 580
 
Starting Slice Number = 0, Ending Slice Number = 892, Middle Reference Slice Number = 580
580
/data/FIBSEM/nuclie_align_ROI3_1700/nuclie30/chromosome/580.jpg
/data/FIBSEM/nuclie_align_ROI3_1700/nuclie30/chromosome/580.jpg
OVER
score_vol_by_100000 {}


In [42]:
print(run_track_chro_track_global.keys())
print(run_track_pattern_track_global.keys())
# print(chro_map_run_0 [7])
# print(run_track_chro_track_global[0])
print(avoid)
print(len(avoid))
print(pattern_avoid)

In [43]:
print(score_vol)
print(score_vol_by_len)

In [44]:
run = 1 #starting from top

print("beg", beg)
print("last", last)

print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")
print(" RUN - ", run)
print("starting from top")

Label_Img={}
chro_track_global={}
pattern_track_global={}
pattern_img = {}
chro_map_global={}
chro_area_global={}
pattern_map_global={}
label_list={}
max_id_c_per_slice=[]
global max_id_c
max_id_c=-1
global limit
limit = 6

order_id = []

mid_flag = True

# find_chromosome(beg, last+1, mid_flag, True, run)
find_chromosome(skip_threshold_top, last+1, mid_flag, True, run)
mid_flag = False
find_chromosome(skip_threshold_top-1, beg-1, mid_flag, False, run)

print("order_id:",order_id)
print(limit)
print(max_id_c)

chro_map_run_1 = chro_map_global
print("*****chro_map_run_1 : ", chro_map_run_1)
chro_area_run_1 = chro_area_global
print("*****chro_area_run_1 : ", chro_area_run_1)
label_list_run1 = label_list
print("*****label_list_run1 : ", label_list_run1)
run_track_chro_track_global[run] = chro_track_global

uniqueValues_1 = set(label_list_run1.keys())
print("uniqueValues",uniqueValues_1)
# new_limit_1 = len(uniqueValues)
# print("new_limit_1", new_limit_1)

chromosomes_to_avoid_run_1 = find_short_chromosomes(run_track_chro_track_global[run])
empty_list = []
print("chromosomes_to_avoid_run_1", chromosomes_to_avoid_run_1)


Label_Img = sort_dictionary(Label_Img)
# save_label_stack(Label_Img, run, False)
save_individual_chromosome(Label_Img, uniqueValues_1, chromosomes_to_avoid_run_1, run, False)

# find_patterns(run, chro_map_run_1, uniqueValues_1, chromosomes_to_avoid_run_1)
# run_track_pattern_track_global[run] = pattern_track_global
# print("pattern_avoid", pattern_avoid)
# #add score
# # length = find_length(pattern_track_global, new_limit_1, chromosomes_to_avoid_run_1)
# # score_run1 = find_score(run_track_chro_track_global[run], chro_area_run_1, new_limit_1, chromosomes_to_avoid_run_1)
# score_v_run1, score_v_by_l_run1, length_run1, volume_run1, legs_length_track_run1, begining_pt_track_run1, ending_pt_track_run1 = find_score(run_track_chro_track_global[run],
#                                                                                                                                              chro_area_run_1, pattern_track_global,
#                                                                                                                                              uniqueValues_1,
#                                                                                                                                              chromosomes_to_avoid_run_1)
# score_vol[run] = score_v_run1
# score_vol_by_len[run] = score_v_by_l_run1


print("DONEEEEEE RUN - ", run)

In [45]:
print(run_track_chro_track_global.keys())
# print(run_track_chro_track_global[0])
# print(run_track_chro_track_global[1])
print(avoid)
print(len(avoid))
print(pattern_avoid)

In [46]:
print(score_vol)
print(score_vol_by_len)

In [47]:
run = 2 #starting from bottom

print("beg", beg)
print("last", last)

print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")
print(" RUN - ", run)
print("starting from bottom")

Label_Img={}
chro_track_global={}
pattern_track_global={}
pattern_img = {}
chro_map_global={}
chro_area_global={}
pattern_map_global={}
label_list={}
max_id_c_per_slice=[]
global max_id_c
max_id_c=-1
global limit
limit = 6

order_id = []

mid_flag = True

# find_chromosome(last, beg-1, mid_flag, False, run)
find_chromosome(last-skip_threshold_bottom, beg-1, mid_flag, False, run)
mid_flag = False
find_chromosome(last-skip_threshold_bottom+1, last+1, mid_flag, True, run)

print("order_id:",order_id)
print(limit)
print(max_id_c)

chro_map_run_2 = chro_map_global
print("*****chro_map_run_2 : ", chro_map_run_2)
chro_area_run_2 = chro_area_global
print("*****chro_area_run_2 : ", chro_area_run_2)
label_list_run2 = label_list
print("*****label_list_run2 : ", label_list_run2)
run_track_chro_track_global[run] = chro_track_global

uniqueValues_2 = set(label_list_run2.keys())
print("uniqueValues", uniqueValues_2)
# new_limit_2 = len(uniqueValues)
# print("new_limit_2", new_limit_2)

chromosomes_to_avoid_run_2 = find_short_chromosomes(run_track_chro_track_global[run])
empty_list = []
print("chromosomes_to_avoid_run_2", chromosomes_to_avoid_run_2)

Label_Img = sort_dictionary(Label_Img)
# save_label_stack(Label_Img, run, False)
save_individual_chromosome(Label_Img, uniqueValues_2, chromosomes_to_avoid_run_2, run, False)

# find_patterns(run, chro_map_run_2, uniqueValues_2, chromosomes_to_avoid_run_2)
# run_track_pattern_track_global[run] = pattern_track_global
# print("pattern_avoid", pattern_avoid)
# #add score
# # length = find_length(pattern_track_global, new_limit_2, chromosomes_to_avoid_run_2)
# # score_run2 = find_score(run_track_chro_track_global[run], chro_area_run_2, new_limit_2, chromosomes_to_avoid_run_2)
# score_v_run2, score_v_by_l_run2, length_run2, volume_run2, legs_length_track_run2, begining_pt_track_run2, ending_pt_track_run2 = find_score(run_track_chro_track_global[run],
#                                                                                                                                              chro_area_run_2, pattern_track_global,
#                                                                                                                                              uniqueValues_2,
#                                                                                                                                              chromosomes_to_avoid_run_2)
# score_vol[run] = score_v_run2
# score_vol_by_len[run] = score_v_by_l_run2

print("DONEEEEEE RUN - ", run)

In [48]:
print(run_track_chro_track_global.keys())
# print(run_track_chro_track_global[0])
# print(run_track_chro_track_global[1])
# print(run_track_chro_track_global[2])
print(avoid)
print(len(avoid))
print(pattern_avoid)

In [49]:
print(score_vol)
print(score_vol_by_len)

In [51]:
print(label_list_run0)
print(label_list_run1)
print(label_list_run2)

temp_0 =label_list_run0
temp_1 =label_list_run1
temp_2 =label_list_run2

In [53]:
#combine labels from the three runs

new_id = 0
for x in range(len(chromosome)):
  print("##########", x)
  n = ntpath.basename(chromosome[x]).split('.')[0]
  n=int(n)
  if n in avoid:
      continue

  list_0 = chro_map_run_0[x]
  list_1 = chro_map_run_1[x]
  list_2 = chro_map_run_2[x]
  print(list_0)
  print(list_1)
  print(list_2)

  for pt in list_0.keys():
    print("point", pt)

    id_0 = list_0[pt]
    id_1 = list_1[pt]
    id_2 = list_2[pt]
    print("id", id_0,id_1,id_2)

    if(label_list_run0[id_0] == label_list_run1[id_1] == label_list_run2[id_2] == None):
      label_list_run0[id_0] = new_id
      label_list_run1[id_1] = new_id
      label_list_run2[id_2] = new_id
      print("assigned")
      print(new_id)
      print(label_list_run0)
      print(label_list_run1)
      print(label_list_run2)
      new_id+=1
      print(".........",new_id)
    elif((label_list_run0[id_0] != None) and (label_list_run1[id_1] != None) and (label_list_run2[id_2] != None)):
      print("elif")
      continue
    else:
      print("else")
      flag_0 = flag_1 = flag_2 = False
      old_0 = old_1 = old_2 = float('inf')
      if(label_list_run0[id_0] != None):
        old_0 = label_list_run0[id_0]
        flag_0 = True
      if(label_list_run1[id_1] != None):
        old_1 = label_list_run1[id_1]
        flag_1 = True
      if(label_list_run2[id_2] != None):
        old_2 = label_list_run2[id_2]
        flag_2 = True
      print("flag_0", flag_0)
      print("flag_1", flag_1)
      print("flag_2", flag_2)
      temp_arr = [old_0, old_1, old_2]
      print(temp_arr)
      new_tag = min(temp_arr)
      label_list_run0[id_0] = label_list_run1[id_1] = label_list_run2[id_2] = new_tag
      print("already")
      print(label_list_run0)
      print(label_list_run1)
      print(label_list_run2)
print(label_list_run0)
print(label_list_run1)
print(label_list_run2)

In [54]:
#Finally run3 from top to bottom, to save masks with contour filled with resulting label after combining
run=3
Final_Image_Stack = {}
pattern_map_global={}
pattern_track_global={}
pattern_img = {}
for x in range(len(chromosome)):
    print(x)
    chromosome_im = Image.open(chromosome[x])
    chromosome_im.load()
    chromo = np.array(chromosome_im)

    chromo = clean_small_obj(chromo,800)

    chromo[chromo<40]=0

    contours_chromosome, hierarchy_chromosome, mask_chro = find_contours(chromo)

    n = ntpath.basename(chromosome[x]).split('.')[0]
    n=int(n)
    if n in avoid:
        Final_Image_Stack[x]=mask_chro
        combined_chro_map_global[x] = 'inf'
        combined_chro_track_global[x] = 'inf'
        continue
    chro_map_temp1 = {}
    chro_map_temp2 = {}
    for i in range(len(contours_chromosome)):
      if cv2.arcLength(contours_chromosome[i],True) > 25 and hierarchy_chromosome[0][i][3]==-1:
        M = cv2.moments(contours_chromosome[i])
        if(M["m00"])==0:
            continue
        cX,cY = int(M["m10"] / M["m00"]),int(M["m01"] / M["m00"])

        points_new=[]
        for l in range(len(contours_chromosome[i])):
          points_new.append(contours_chromosome[i][l][0])
        points_new = np.array(points_new)
        cv2.drawContours(mask_chro,[points_new],-1,((label_list_run1[chro_map_run_1[x][(cX,cY)]])+1), thickness = cv2.FILLED)
        chro_map_temp1[cX,cY] = [label_list_run1[chro_map_run_1[x][(cX,cY)]]]
        chro_map_temp2[cX,cY] = label_list_run1[chro_map_run_1[x][(cX,cY)]]
        # cv2.fillPoly(mask_both , pts=[points], color=colors[chro_map_local[(cX,cY)][0]])
    Final_Image_Stack[x]=mask_chro
    combined_chro_map_global[x] = chro_map_temp2
    chro_track_temp = get_chromosome_tracking(chro_map_temp1)
    combined_chro_track_global[x] = chro_track_temp

print("%%%%%%%%%%%%%%%%%%%%%%%combined_chro_map_global", combined_chro_map_global)
print("%%%%%%%%%%%%%%%%%%%%%%%combined_chro_track_global", combined_chro_track_global)
combined_uniqueValues = set(label_list_run1.values())
# new_limit = len(uniqueValues)


print("uniqueValues",combined_uniqueValues)
# print("new_limit", new_limit)

##### remove all short length chromosomes
combined_chromosomes_to_avoid = find_short_chromosomes(combined_chro_track_global)
print("combined_chromosomes_to_avoid", combined_chromosomes_to_avoid)


# # print(Final_Image_Stack.keys())
# sort_dictionary(Final_Image_Stack)
# # print(Final_Image_Stack.keys())
# save_label_stack(Final_Image_Stack, run, False)
save_individual_chromosome(Final_Image_Stack, combined_uniqueValues, combined_chromosomes_to_avoid, run, False)
# find_patterns(run, combined_chro_map_global, combined_uniqueValues, combined_chromosomes_to_avoid)
# run_track_pattern_track_global[run] = pattern_track_global
# print("pattern_avoid", pattern_avoid)
# #add score
# # length = find_length(pattern_track_global, new_limit, combined_chromosomes_to_avoid)
# # score_combined = find_score(combined_chro_track_global, chro_area_run_1, new_limit, combined_chromosomes_to_avoid)
# score_v_combined, score_v_by_l_combined, length_combined, volume_combined, legs_length_track_combined, begining_pt_track_combined, ending_pt_track_combined = find_score(combined_chro_track_global,
#                                                                                                                                              chro_area_run_1, pattern_track_global,
#                                                                                                                                              combined_uniqueValues,
#                                                                                                                                              combined_chromosomes_to_avoid)
# score_vol[run] = score_v_combined
# score_vol_by_len[run] = score_v_by_l_combined
# print(" ")

print("OOOOOOOOOVVVVVVEEEEEEERRRRRRRR")


In [55]:
# print("score_vol", score_vol)
# print(" ")
# print(" ")
# score_vol_by_100000={}
# for run_no in score_vol:
#     score_vol_by_100000[run_no] = score_vol[run_no]/100000
# print("score_vol_by_100000", score_vol_by_100000)
# print("score_vol_by_len", score_vol_by_len)
# print(" ")


In [60]:
#stop writting to txt file
sys.stdout = sys.__stdout__

In [61]:
print("OVER")
# print("score_vol_by_100000", score_vol_by_100000)

In [ ]:
# print(order_id)
# print(limit)
# print(max_id_c)